# 测试Random

In [4]:
from types import GenericAlias
from dataclasses import dataclass
from abc import abstractmethod, ABC
from __future__ import annotations
from typing import Any, Callable, Tuple, TypeVar

S = TypeVar("S")
T = TypeVar("T")


class RNG(ABC):
    
    @abstractmethod
    def next_int(self) -> Tuple[int, RNG]:
        pass


@dataclass
class SimpleRNG(RNG):
    seed: int
    def next_int(self) -> Tuple[int, RNG]:
        new_seed = (self.seed * 0x5DEECE66D + 0xB) & 0xFFFFFFFFFFFF
        next_RNG = SimpleRNG(new_seed)
        n = int(new_seed >> 16)
        return (n, next_RNG)

In [10]:
Rand = GenericAlias(Callable[[RNG], Tuple[S, RNG]], S)

rand_int: Rand[int] = SimpleRNG.next_int

def unit(s: S) -> Rand[S]:
    return lambda rng: (s, rng)

def map(rand: Rand[S], f: Callable[[S], T]) -> Rand[T]:
    def helper(rng: RNG):
        res, rng2 = rand(rng)
        return (f(res), rng2)
    return helper

In [22]:
rand_non_negative_int: Rand[int] = map(rand_int, abs)
rand_non_negative_even: Rand[int] = map(rand_int, lambda x: x - x % 2)